In [24]:
import numpy as np
import pandas as pd

In [25]:
tab=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [26]:
df=tab[0]
df.columns=[' PostalCode', 'Borough', 'Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


As we want only rows where Borough should not be (not assigned)  we  will only select dataframe where    Borough!='Not Assigned'

In [27]:
df_neigh=df[df['Borough']!='Not assigned']
df_neigh.reset_index(inplace=True)
df_neigh.drop(columns='index',inplace=True)
df_neigh.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


We will check whether Duplicater values of Postal Code exists or not

In [28]:
bool_series = df_neigh.iloc[:,0].duplicated(keep = False) 
df_neigh[bool_series]

,PostalCode,Borough,Neighborhood


As we need to drop Not Assign rows we will compare Neighbour = Not assigned or not

In [29]:
df_neigh[df_neigh['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood


In [30]:
df_neigh.shape

(103, 3)

In [31]:
geo_data=pd.read_csv('Geospatial_Coordinates.csv')


In [32]:
geo_data[0:5]

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [49]:
lat=[]
long=[]
for index , row in geo_data.iterrows() :
    for j, r in  df_neigh.iterrows() :
        if row[0]==r[0] :
            lat.append(row[1])
            long.append(row[2])
            continue
            

In [50]:
df_neigh['Latitude']= lat
df_neigh['Longitude']= long

C:\Users\digvi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\digvi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [53]:
df_neigh.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476


In [51]:
df_neigh[df_neigh['Neighborhood']=='Central Bay Street']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
24,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259


In [52]:
geo_data[geo_data['Postal Code']=='M5G']

,Postal Code,Latitude,Longitude
57,M5G,43.657952,-79.387383
